# Convolutional Neural Network

### Importing the libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
from keras.preprocessing import image

In [ ]:
tf.__version__

## Part 1 - Data Preprocessing

### Preprocessing the Training set

#### Image Augmentaion:
To avoid overfitting, we are going to do some geometrical transforamtions to the images (like shifitng,rotating, zoom ins/outs...). We are aumneting the diversity of the input images

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,  # feature scaling, put allpixel betweem 0-1 range
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64), # size of image to be fed to the nn
                                                 batch_size = 32,
                                                 class_mode = 'binary') # binary or categorical

### Preprocessing the Test set
Be carefull no geometrical transformation on the test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64), # the same as the training set
                                            batch_size = 32,
                                            class_mode = 'binary')

## Part 2 - Building the CNN

### Initialising the CNN:
Wwe add layers so we wanat the sequential class

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution
- filters -> no of feature detectors to apply to the images, 32 is a common num but we can experiment
- kernel_size -> so filter is a squared array to apply to the image, so the kernel size is the dimensions of the array, so if 3 this means 3x3 filter
- activation --> relu for the cnn layer to add non linearity to the data
- input_shape--> [64x64x3] (where 3 as we have coloured images, do RGB), if it was black/white images it would have been [64x64x1]

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling
- Max pooling
- pool_size: the frame based on which we get the amx value. If 2 thsi means a 2x2 frame
- strides = how much we move the frame every time

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer
- The input shape parameter is necessary only in the first convolution layer 

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening
Put all in one dimensional vector to send it ot the fully connected layer

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection
- units: number of hidden neurons
- activation: relu to add non linearity

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set
Tried 10, 15 epochs but wasn't converging so tried more 

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

## Part 4 - Making a single prediction

In [ ]:
# get test unseen example
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64)) #same size as the training
test_image = image.img_to_array(test_image)

# add dimension corresponding to the batch, fake dimesion
test_image = np.expand_dims(test_image, axis = 0) 
result = cnn.predict(test_image)

print(training_set.class_indices) # to check which class corresponds to which class

if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

In [ ]:
print(prediction)